In [ ]:
## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="6G")
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
# %aimport islets

from IPython.display import display
import pandas as pd

from islets.Recording import Recording, saveMovie
from islets.Regions import Regions
from islets.numeric import power_spectrum, rebin
from islets.utils import show_movie, saveRois
from islets.Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph

from caiman import movie as cmovie

from matplotlib.colors import LogNorm

In [ ]:
# from copy import deepcopy
fracSaturTh = .05
cmap = plt.cm.Greys
cmap.set_bad("lime")
pd.set_option('display.max_rows', 100)

In [ ]:
# rec = Recording("/data/Sandra/2020/2020_06_24/Experiment56a.lif")
# rec = Recording("/data/MB2020_lifs/2012/2012_01_31/Pancreas_Islet_OGB13.lif")

In [ ]:
# recs = [Recording("/data/Sandra/2020/2020_06_22/Series%03i.nd2"%i) for i in range(9,14)]

# for rec in recs:
#     display(rec.metadata)

In [ ]:
rec = Recording("/data/Sandra/2020/2020_06_22/Series011.nd2")

In [ ]:
ser = "all"

In [ ]:
rec.import_series(ser)

In [ ]:
metadata = rec.Series[ser]['metadata']

In [ ]:
metadata

In [ ]:
movie = cmovie(rec.Series[ser]['data'],fr = metadata.Frequency)

In [ ]:
dmovie = np.diff(movie.astype(np.in16))

In [ ]:
plt.plot(dmovie.mean((1,2)))

In [ ]:
movie.shape

In [ ]:
movie.fr

In [ ]:
meanImage = movie.mean(0)

In [ ]:
plt.imshow(meanImage, norm=LogNorm())

In [ ]:
Nsatur = (movie==movie.max()).sum(0)

In [ ]:
plt.imshow(Nsatur.astype(float),
#            norm=LogNorm()
          )
plt.colorbar()

In [ ]:
plt.rcParams["animation.embed_limit"] = 200

In [ ]:
# saveMovie(movie, "embed")

### Wiggles?

In [ ]:
rwig = Regions(meanImage, gSig_filt=(3,))

In [ ]:
freq = metadata["Frequency"]

In [ ]:
n = 20
modFreq = []
for i in rwig.df.index:
    point = rwig.df.loc[i,"peak"]
    x = movie[:,point[0],point[1]]
    f,p = power_spectrum(x, fr=freq)
    f,p = f[f>1],np.abs(p[f>1])
    f = rebin(f,n)
    p = rebin(p,n)
    rwig.df.loc[i,"peakFreq"] = f[np.argmax(p)]
    rwig.df.loc[i,"peakProminence"] = np.max(p)

In [ ]:
tmpdf = rwig.df[["peakValue","peakFreq","peakProminence"]].copy()
tmpdf["log10 mean"] = np.log10(tmpdf["peakValue"])
tmpdf["log10 peakProminence"] = np.log10(tmpdf["peakProminence"])
del tmpdf["peakProminence"], tmpdf["peakValue"]
pd.plotting.scatter_matrix(tmpdf, figsize=(10,10));

### import pickle

In [ ]:
sorted(os.listdir(os.path.join(rec.path+"_analysis",ser)))

In [ ]:
pickleFile = os.path.join(rec.path+"_analysis",ser,"8_rois.pkl")
with open(pickleFile,"rb") as f:
    regions = pickle.load(f)

In [ ]:
regions.update()
regions.detrend_traces()

In [ ]:
regions.plotEdges()

In [ ]:
regions.infer_gain(plot=True)

In [ ]:
regions.gain

In [ ]:
movie.shape, movie.dtype, movie.max()

In [ ]:
# saveMovie(movie[:1000], "embed")

In [ ]:
from islets.numeric import get_sep_th

In [ ]:
intTh = np.exp(get_sep_th(np.log(regions.df.interest), plot=True))

In [ ]:
# show_movie(movie,
#            NTimeFrames=180,
#            additionalPlot=lambda ax_:regions.plotEdges(ax=ax_, image=False, color="navy",ix)
#           )

In [ ]:
plt.plot(regions.df.trace[0])